In [1]:
import sys
import spacy
import os

from dotenv import load_dotenv
project_dir = os.getenv("PROJECT_PATH")

sys.path.insert(1, project_dir + '/utils/')
sys.path.insert(2, project_dir + '/utils/neo4j/')

import kb_util

from connection import Neo4jConnection
import query as query

def search_matching_skill(skill, skills, threshold=0.5):    
    
    search = nlp(skill)    
    high_score = 0
    term = ""

    for elem in skills:
        match = nlp(elem)
            
        score = search.similarity(match)
        
        if score > high_score:
            high_score = score
            term = match
    
    if high_score > threshold:
        return search, term, high_score
    else:
        return None, None, 0  


def clean_skill_list(found_skills, skills, threshold=0.5):
    
    found_skills = [x.lower() for x in found_skills]
    skills = [x.lower() for x in skills]    
    new_set= set()
    
    for found_skill in found_skills:    
   
        search, term, high_score = search_matching_skill(found_skill, skills, threshold)
        if high_score > 0:
            if high_score == 1.0:
                #print("Found in KB", found_skill)           
                new_set.add(found_skill)
            else:
                #print(search, "<->", term, "==", high_score)              
                new_set.add(term.text)   

    return new_set


nlp = spacy.load("en_core_web_lg")  # make sure to use larger package!

conn = Neo4jConnection(uri=os.getenv("DB_URI"), 
                    user=os.getenv("DB_USERNAME"),              
                    pwd=os.getenv("DB_PASSWORD"))

skills = query.get_all_skills(conn)

In [32]:
found_skills = ["Python 3.2", "MS Word", "Microsoft Word", "c++"]

print(clean_skill_list(found_skills, skills, 0.6))

C:\Users\tom\AppData\Local\Temp\ipykernel_15844\2653724992.py:21: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  score = search.similarity(match)


{'c++', 'python', 'word'}
